# 7. Оконные функции

**Задание №1**

Примените оконные функции к таблице **products** и с помощью ранжирующих функций упорядочьте все товары по цене — от самых дорогих к самым дешёвым. Добавьте в таблицу следующие колонки:

Колонку **product_number** с порядковым номером товара (функция **ROW_NUMBER**).
Колонку **product_rank** с рангом товара с пропусками рангов (функция **RANK**).
Колонку **product_dense_rank** с рангом товара без пропусков рангов (функция **DENSE_RANK**).
Не забывайте указывать в окне сортировку записей — без неё ранжирующие функции могут давать некорректный результат, если таблица заранее не отсортирована. Деление на партиции внутри окна сейчас не требуется. Сортировать записи в результирующей таблице тоже не нужно.

Поля в результирующей таблице: **product_id**, **name**, **price**, **product_number**, **product_rank**, **product_dense_rank**

После того как решите задачу, посмотрите, что в итоге получилось, и проанализируйте результат.

```sql
SELECT product_id,
       name,
       price,
       ROW_NUMBER() OVER (ORDER BY price DESC) as product_number,
       RANK() OVER (ORDER BY price DESC) as product_rank,
       DENSE_RANK() OVER (ORDER BY price DESC) as product_dense_rank
FROM   products
```

**Задание №2**

Примените оконную функцию к таблице **products** и с помощью агрегирующей функции в отдельной колонке для каждой записи проставьте цену самого дорогого товара. Колонку с этим значением назовите **max_price**.

Затем для каждого товара посчитайте долю его цены в стоимости самого дорогого товара — просто поделите одну колонку на другую. Полученные доли округлите до двух знаков после запятой. Колонку с долями назовите **share_of_max**.

Выведите всю информацию о товарах, включая значения в новых колонках. Результат отсортируйте сначала по убыванию цены товара, затем по возрастанию **id** товара.

Поля в результирующей таблице: **product_id**, **name**, **price**, **max_price**, **share_of_max**
```sql

SELECT product_id,
       name,
       price,
       MAX(price) OVER () AS max_price,
       ROUND(price / MAX(price) OVER (), 2) AS share_of_max
FROM   products
ORDER BY price DESC, product_id
```

**Задание №3**

Примените две оконные функции к таблице **products**. Одну с агрегирующей функцией **MAX**, а другую с агрегирующей функцией **MIN** — для вычисления максимальной и минимальной цены. Для двух окон задайте инструкцию **ORDER BY** по убыванию цены. Поместите результат вычислений в две колонки **max_price** и **min_price**.

Выведите всю информацию о товарах, включая значения в новых колонках. Результат отсортируйте сначала по убыванию цены товара, затем по возрастанию **id** товара.

Поля в результирующей таблице: **product_id**, **name**, **price**, **max_price**, **min_price**

После того как решите задачу, проанализируйте полученный результат и подумайте, почему получились именно такие расчёты. При необходимости вернитесь к первому шагу и ещё раз внимательно ознакомьтесь с тем, как работает рамка окна при указании сортировки.
```sql
SELECT product_id,
       name,
       price,
       MAX(price) OVER (ORDER BY price DESC) AS max_price,
       MIN(price) OVER (ORDER BY price desc) AS min_price
FROM   products
ORDER BY price DESC, product_id
```

**Задание №4**

Сначала на основе таблицы **orders** сформируйте новую таблицу с общим числом заказов по дням. При подсчёте числа заказов не учитывайте отменённые заказы (их можно определить по таблице **user_actions**). Колонку с днями назовите **date**, а колонку с числом заказов — **orders_count**.

Затем поместите полученную таблицу в подзапрос и примените к ней оконную функцию в паре с агрегирующей функцией **SUM** для расчёта накопительной суммы числа заказов. Не забудьте для окна задать инструкцию **ORDER BY** по дате.

Колонку с накопительной суммой назовите **orders_cum_count**. В результате такой операции значение накопительной суммы для последнего дня должно получиться равным общему числу заказов за весь период.

Сортировку результирующей таблицы делать не нужно.

Поля в результирующей таблице: **date**, **orders_count**, **orders_cum_count**
```sql
-- фильтруем, убирая отмененные заказы
WITH
t1 AS (SELECT *
       FROM   orders
       WHERE  order_id not in (SELECT order_id
                               FROM   user_actions
                               WHERE  action = 'cancel_order'))
-- отбираем необходимые колонки и в оконной функции подсчитываем количество заказов для каждой даты
SELECT date,
       orders_count,
       SUM(orders_count) OVER (ORDER BY date)::integer AS orders_cum_count
       -- приводим creation_time к формату даты, подсчитываем количество заказов
FROM   (SELECT creation_time::date AS date,
               COUNT(order_id) AS orders_count
        FROM   t1
        GROUP BY date) t2
```

**Задание №5**

Для каждого пользователя в таблице **user_actions** посчитайте порядковый номер каждого заказа.

Для этого примените оконную функцию **ROW_NUMBER** к колонке с временем заказа. Не забудьте указать деление на партиции по пользователям и сортировку внутри партиций. Отменённые заказы не учитывайте.

Новую колонку с порядковым номером заказа назовите order_number. Результат отсортируйте сначала по возрастанию **id** пользователя, затем по возрастанию порядкового номера заказа.

Добавьте в запрос оператор **LIMIT** и выведите только первые 1000 строк результирующей таблицы.

Поля в результирующей таблице: **user_id**, **order_id**, **time**, **order_number**

```sql
SELECT user_id,
       order_id,
       time,
       ROW_NUMBER() OVER (PARTITION BY user_id
                          ORDER BY order_id) AS order_number
FROM   user_actions
WHERE  order_id NOT IN (SELECT order_id
                        FROM   user_actions
                        WHERE  action = 'cancel_order')
ORDER BY user_id, order_number 
LIMIT 1000
```

**Задание №6**

Дополните запрос из предыдущего задания и с помощью оконной функции для каждого заказа каждого пользователя рассчитайте, сколько времени прошло с момента предыдущего заказа. 

Для этого сначала в отдельном столбце с помощью **LAG** сделайте смещение по столбцу **time** на одно значение назад. Столбец со смещёнными значениями назовите **time_lag**. Затем отнимите от каждого значения в колонке time новое значение со смещением (либо можете использовать уже знакомую функцию **AGE**). Колонку с полученным интервалом назовите **time_diff**. Менять формат отображения значений не нужно, они должны иметь примерно следующий вид:

3 days, 12:18:22

По-прежнему не учитывайте отменённые заказы. Также оставьте в запросе порядковый номер каждого заказа, рассчитанный на прошлом шаге. Результат отсортируйте сначала по возрастанию **id** пользователя, затем по возрастанию порядкового номера заказа.

Добавьте в запрос оператор **LIMIT** и выведите только первые 1000 строк результирующей таблицы.

Поля в результирующей таблице: **user_id**, **order_id**, **time**, **order_number**, **time_lag**, **time_diff**

```sql
SELECT user_id,
       order_id,
       time,
       ROW_NUMBER() OVER (PARTITION BY user_id
                          ORDER BY order_id) AS order_number,
       LAG(time, 1) OVER (PARTITION BY user_id
                          ORDER BY order_id) AS time_lag,
       time - LAG(time, 1) OVER (PARTITION BY user_id
                                 ORDER BY order_id) AS time_diff
FROM   user_actions
WHERE  order_id NOT IN (SELECT order_id
                        FROM   user_actions
                        WHERE  action = 'cancel_order')
ORDER BY user_id, order_number 
LIMIT 1000
```

**Задание №7**

На основе запроса из предыдущего задания для каждого пользователя рассчитайте, сколько в среднем времени проходит между его заказами. Посчитайте этот показатель только для тех пользователей, которые за всё время оформили более одного неотмененного заказа.

Среднее время между заказами выразите в часах, округлив значения до целого числа. Колонку со средним значением времени назовите **hours_between_orders**. Результат отсортируйте по возрастанию **id** пользователя.

Добавьте в запрос оператор **LIMIT** и включите в результат только первые 1000 записей.

Поля в результирующей таблице: **user_id**, **hours_between_orders**


```sql
SELECT user_id,
       AVG(time_diff)::integer AS hours_between_orders
FROM   (SELECT user_id,
               extract(epoch
        FROM   (time - LAG(time, 1) OVER (PARTITION BY user_id
        ORDER BY order_id))) /3600 AS time_diff
        FROM   user_actions
        WHERE  order_id not in (SELECT order_id
                                FROM   user_actions
                                WHERE  action = 'cancel_order')) AS t1
WHERE  time_diff IS NOT NULL
GROUP BY user_id
ORDER BY user_id
LIMIT 1000
```

**Задание №8**

Сначала на основе таблицы **orders** сформируйте новую таблицу с общим числом заказов по дням. Вы уже делали это в одной из предыдущих задач. При подсчёте числа заказов не учитывайте отменённые заказы (их можно определить по таблице **user_actions**). Колонку с числом заказов назовите **orders_count**.

Затем поместите полученную таблицу в подзапрос и примените к ней оконную функцию в паре с агрегирующей функцией **AVG** для расчёта скользящего среднего числа заказов. Скользящее среднее для каждой записи считайте по трём предыдущим дням. Подумайте, как правильно задать границы рамки, чтобы получить корректные расчёты.

Полученные значения скользящего среднего округлите до двух знаков после запятой. Колонку с рассчитанным показателем назовите **moving_avg**. Сортировку результирующей таблицы делать не нужно.

Поля в результирующей таблице: **date**, **orders_count**, **moving_avg**
```sql
SELECT date,
       orders_count,
       ROUND(AVG(orders_count) OVER (ORDER BY date ROWS BETWEEN 3 preceding AND 1 preceding), 2) AS moving_avg 
FROM(SELECT creation_time::date AS date,
            COUNT (DISTINCT order_id) AS orders_count
     FROM   orders
     WHERE  order_id not in (SELECT order_id
                             FROM   user_actions
                             WHERE  action = 'cancel_order')
                             GROUP BY date)t;
```

**Задание №9**

Отметьте в отдельной таблице тех курьеров, которые доставили в сентябре 2022 года заказов больше, чем в среднем все курьеры.

Сначала для каждого курьера в таблице **courier_actions** рассчитайте общее количество доставленных в сентябре заказов. Затем в отдельном столбце с помощью оконной функции укажите, сколько в среднем заказов доставили в этом месяце все курьеры. После этого сравните число заказов, доставленных каждым курьером, со средним значением в новом столбце. Если курьер доставил больше заказов, чем в среднем все курьеры, то в отдельном столбце с помощью **CASE** укажите число 1, в противном случае укажите 0.

Колонку с результатом сравнения назовите **is_above_avg**, колонку с числом доставленных заказов каждым курьером — **delivered_orders**, а колонку со средним значением — **avg_delivered_orders**. При расчёте среднего значения округлите его до двух знаков после запятой. Результат отсортируйте по возрастанию **id** курьера.

Поля в результирующей таблице: **courier_id**, **delivered_orders**, **avg_delivered_orders**, **is_above_avg**

```sql
SELECT courier_id,
       COUNT(order_id) AS delivered_orders,
       ROUND(AVG(COUNT(order_id)) OVER(), 2) AS avg_delivered_orders,
       CASE WHEN COUNT(order_id) > AVG(COUNT(order_id)) OVER() THEN 1
            WHEN COUNT(order_id) < AVG(COUNT(order_id)) OVER() THEN 0 END AS is_above_avg
FROM   courier_actions
WHERE  action = 'deliver_order'
   AND date_part('month', time) = 9
GROUP BY courier_id
ORDER BY courier_id
```

**Задание №10**

По данным таблицы **user_actions** посчитайте число первых и повторных заказов на каждую дату.

Для этого сначала с помощью оконных функций и оператора **CASE** сформируйте таблицу, в которой напротив каждого заказа будет стоять отметка «Первый» или «Повторный» (без кавычек). Для каждого пользователя первым заказом будет тот, который был сделан раньше всего. Все остальные заказы должны попасть, соответственно, в категорию «Повторный». Затем на каждую дату посчитайте число заказов каждой категории.

Колонку с типом заказа назовите **order_type**, колонку с датой — **date**, колонку с числом заказов — **orders_count**.

В расчётах учитывайте только неотменённые заказы.

Результат отсортируйте сначала по возрастанию даты, затем по возрастанию значений в колонке с типом заказа.

Поля в результирующей таблице: **date**, **order_type**, **orders_count**


```sql
-- подсчитываем заказы, которые далее будут сгруппированы по дате и типу
SELECT time::date AS date,
       order_type,
       COUNT(order_id) AS orders_count
       -- находим минимальную дату для каждого юзера, делим на категории и фильтруем
FROM   (SELECT user_id,
               order_id,
               time,
               CASE WHEN time = MIN(time) OVER (PARTITION BY user_id) THEN 'Первый'
                    ELSE 'Повторный' END AS order_type
        FROM   user_actions
        WHERE  order_id not in (SELECT order_id
                                FROM   user_actions
                                WHERE  action = 'cancel_order'))t1
GROUP BY date, order_type
ORDER BY date, order_type
```

**Задание №11**
x
```sql
x
```
